In [1]:
import utilities, myanimelistAPI, jikan4pyAPI, json
import requests as r
import datetime as dt
from utilities import print_bot

In [2]:
malID_to_aniListID = {}
mal_api = myanimelistAPI.MyAnimeListAPI(open("mal.key").read())
jikan_api = jikan4pyAPI.JikanAPI()

In [3]:
def malIDtoAniListID(anime_id):
	"""
	Converts the MAL ID to AniList ID
	using GraphQL
	"""
	if anime_id in malID_to_aniListID:
		return malID_to_aniListID[anime_id]

	query = """query
		($id: Int, $type: MediaType) {
			Media(idMal: $id, type: $type) {
				id}
			}
		"""
	variables = {'id': anime_id, 'type': 'ANIME'}
	url = 'https://graphql.anilist.co'
	response = r.post(url, json={'query': query, 'variables': variables})
	malID_to_aniListID[anime_id] = int(response.json()['data']['Media']['id'])
	return malID_to_aniListID[anime_id]

def timeTillNextEpisode(anime_id):
	"""
	Returns the time till the next episode of the anime airs
	"""
	query = """query ($id: Int) {
					Media(idMal: $id, type: ANIME) {
						id
						nextAiringEpisode {
						timeUntilAiring
						}
					}
				}
	"""
	variables = {'id': anime_id, 'type': 'ANIME'}
	url = 'https://graphql.anilist.co'
	data = r.post(url, json={'query': query, 'variables': variables}).json()

	try: # if parsing fails, then the anime is not airing
		time_data = data['data']['Media']['nextAiringEpisode']
	except:
		time_data = None

	if time_data is not None:
		seconds = time_data['timeUntilAiring']
	else:
		return {"days": 0,
				"hours": 0,
				"minutes": 0,
				"seconds": 0,
				"total_seconds": 0,
				"isNull": True}

	time = dt.timedelta(seconds=seconds)
	return {"days": time.days,
			"hours": time.seconds//3600,
			"minutes": (time.seconds//60) % 60,
			"seconds": time.seconds % 60,
			"total_seconds": seconds,
			"isNull": False}
 
def isAnimeAiring(anime_id, api="mal"):
	"""
	Checks if the anime is airing
	using the MyAnimeList API (atleast more updated then the Jikan API due to less caching)
	"""
	if api == "mal":
		try:
			result = mal_api.getAnimeByID(anime_id)
			return (result['status'] != "finished_airing")
		except Exception as e:
			print_bot(
				f"Error: Mal API ID check failed, so switching to Jikan API.\n'{e}'")
			result = jikan_api.getAnimeByID(anime_id)
			return result["status"] != "Finished Airing"
	else:
		result = jikan_api.getAnimeByID(anime_id)
		return result["status"] != "Finished Airing"

In [5]:
animes = jikan_api.searchAnime(anime_name="Jujutsu Kaisen")
anime_titles = [anime["title"] for anime in animes]

In [67]:
import pickledb

In [68]:
db = pickledb.load("database.json", False)

In [69]:
db.set("1234545", {"A":"B"})

True

In [85]:
db.get("1234545").update({"D":"E"})

In [93]:
db.get("1234545").pop("D")

'E'

In [94]:
db.get("1234545")

{'A': 'B', 'B': 'C'}

In [98]:
[str(value) for value in db.get("1234545").values()]

['B', 'C']

In [91]:
[(el,db.get("1234545")[el]) for el in db.get("1234545")]

[('A', 'B'), ('B', 'C'), ('D', 'E')]

In [88]:
print(["\n".join(str(value) for value in db.get("1234545").values())][0])

B
C
E


In [72]:
db.dump()

True